In [1]:
import numpy as np
from itertools import chain, combinations
from collections import OrderedDict

In [2]:
class Itemset:
    def __init__(self, items: list):
        self.items = items
        self.size = len(items)
    
    def getItems(self):
        return self.items
    def getSize(self):
        return self.size

In [3]:
# read all transactions from comma-separated file
def read_transactions(filename):
    transactions = []
    with open(filename, 'r') as f:
        for line in f:
            transactions.append(line.strip().split(','))
    return transactions

In [4]:
# convert all elements in txn from string to int
def convert_to_int(txns):
    for i in range(len(txns)):
        for j in range(len(txns[i])):
            txns[i][j] = int(txns[i][j])
        txns[i].sort()
    return txns


In [5]:
def convert_to_objects(txn):
    ret = []
    for i in range(len(txn)):
        ret.append(Itemset(txn[i]))
    return ret

In [6]:
def vec2str(vec):
    ret = ''
    for i in range(len(vec)):
        ret += str(vec[i]) + ' '
    ret = ret.rstrip()
    return ret

def str2vec(str: str):
    ret = []
    for i in range(len(str.split(' '))):
        ret.append(int(str.split(' ')[i]))
    return ret

def count_spaces(str: str):
    return str.count(' ')

In [7]:
txns = read_transactions('data/grocery_500.txt')
txns = convert_to_int(txns)

txns.sort(key=len, reverse=True)
txns = txns[:100]

txns_itemsets = convert_to_objects(txns)

In [8]:
vec2str("12 15")

'1 2   1 5'

In [9]:
txns
# txn_itemsets # find a better name for this

[[17, 18, 23, 38, 52, 113, 135, 138],
 [4, 16, 18, 39, 49, 68, 79, 80],
 [0, 1, 6, 8, 17, 24, 52, 58],
 [1, 4, 8, 9, 13, 33, 49],
 [0, 17, 21, 37, 55, 67, 68],
 [1, 3, 8, 17, 36, 39, 137],
 [3, 10, 25, 37, 59, 82],
 [3, 17, 22, 38, 70, 127],
 [10, 15, 37, 39, 95, 162],
 [10, 23, 44, 69, 77, 135],
 [3, 21, 43, 58, 59, 60],
 [0, 4, 21, 65, 70],
 [2, 4, 8, 72, 128],
 [10, 11, 64, 79, 129],
 [1, 10, 11, 18, 22]]

In [10]:
# sample dictionary
d = {'1': 24, '10': 86, '5': 33, '20': 34, '1 20': 34, '5 7': 36, '9 10': 37,'1 21': 37}
d = {'1': 2, '10': 2, '5': 2, '20': 1, '1 20': 1, '5 7': 1, '9 10': 1,'1 21': 1}
print(d.items())
# SCO = decreasing by count, decreasing by support, lexicographic
# Try to test more thoroughly
# lexicographic
# d = { k:v for k,v in sorted(d.items(), key=lambda x: x[0])}
d = { k:v for k,v in sorted(d.items(), key=lambda x: str2vec(x[0]))}
# by support
d = { k:v for k,v in sorted(d.items(), key=lambda x: x[1], reverse=True)}
#by count
d = { k:v for k,v in sorted(d.items(), key=lambda x: count_spaces(x[0]), reverse=True)}
print(d)


dict_items([('1', 2), ('10', 2), ('5', 2), ('20', 1), ('1 20', 1), ('5 7', 1), ('9 10', 1), ('1 21', 1)])
{'1 20': 1, '1 21': 1, '5 7': 1, '9 10': 1, '1': 2, '5': 2, '10': 2, '20': 1}


In [11]:
# def sortSCadO(d):
#     # Standard Candidate Order =  decreasing by support, decreasing by count, lexicographic
#     # print(d)
#     # sort items by their frequency
#     d = { k:v for k,v in sorted(d.items(), key=lambda x: str2vec(x[0]))}
#     #by count
#     d = { k:v for k,v in sorted(d.items(), key=lambda x: count_spaces(x[0]), reverse=True)}
#     # by support(X) = number of transactions that contain X
#     d = { k:v for k,v in sorted(d.items(), key=lambda x: x[1], reverse=True)}
#     print("In SCADO")
#     print(d)

#     return d

In [12]:
def sortSCadO(d):
    # Standard Candidate Order =  decreasing by support, decreasing by count, lexicographic
    # sort items by their frequency
    d = OrderedDict(sorted(d.items(), key=lambda x: str2vec(x[0])))
    #by count
    d = OrderedDict(sorted(d.items(), key=lambda x: count_spaces(x[0]), reverse=True))
    # by support(X) = number of transactions that contain X
    d = OrderedDict(sorted(d.items(), key=lambda x: x[1], reverse=True))
    # print("In SCADO")
    # print(d)

    return d

In [13]:
# def sortSCovO(d):
#     # sort items by their frequency
#     d = { k:v for k,v in sorted(d.items(), key=lambda x: str2vec(x[0]))}
#     # by support
#     d = { k:v for k,v in sorted(d.items(), key=lambda x: x[1], reverse=True)}
#     #by count
#     d = { k:v for k,v in sorted(d.items(), key=lambda x: count_spaces(x[0]), reverse=True)}
#     # print(d)
#     return d

In [14]:
def sortSCovO(d):
    # sort items by their frequency
    d = OrderedDict(sorted(d.items(), key=lambda x: str2vec(x[0])))
    # by support
    d = OrderedDict(sorted(d.items(), key=lambda x: x[1], reverse=True))
    #by count
    d = OrderedDict(sorted(d.items(), key=lambda x: count_spaces(x[0]), reverse=True))
    # print(d)
    return d

In [15]:
def getStandardCodeTable(txns: list):
    """
    <algorithm 1>
    
    Get the standard code table from the database.

    @param txns: database / list of transactions
    @return: standard code table SCT
    SCT = { itemName:(code,frequency) , ...}
    """
    # get the unique items in transactions
    d = OrderedDict()
    for tx in txns:
        for item in tx:
            key = vec2str([item])
            if(key not in d):
                d[key] = 1
            else:
                d[key]+=1
    
    # sort items by their frequency
    d = sortSCovO(d)

    # get the standard code table, we don't need the actual code rn
    code_table = OrderedDict()
    # actual_code = 0
    for key, value in d.items():
        code_table[key] = value
        # code_table[key] = (actual_code,value)
        # actual_code+=1

    return code_table


global SCT
SCT = getStandardCodeTable(txns)
# CT = SCT.copy()

In [16]:
SCT

OrderedDict([('10', 5),
             ('17', 5),
             ('1', 4),
             ('3', 4),
             ('4', 4),
             ('8', 4),
             ('0', 3),
             ('18', 3),
             ('21', 3),
             ('37', 3),
             ('39', 3),
             ('11', 2),
             ('22', 2),
             ('23', 2),
             ('38', 2),
             ('49', 2),
             ('52', 2),
             ('58', 2),
             ('59', 2),
             ('68', 2),
             ('70', 2),
             ('79', 2),
             ('135', 2),
             ('2', 1),
             ('6', 1),
             ('9', 1),
             ('13', 1),
             ('15', 1),
             ('16', 1),
             ('24', 1),
             ('25', 1),
             ('33', 1),
             ('36', 1),
             ('43', 1),
             ('44', 1),
             ('55', 1),
             ('60', 1),
             ('64', 1),
             ('65', 1),
             ('67', 1),
             ('69', 1),
             ('72', 1),

In [17]:
def support(X):
    # [TODO] How do we get SCT here properly?
    # look into OOP based approach
    return SCT[X]/len(txns)

In [18]:
def compareSCO(X1, X2):
    '''
    Compare two transactions in the **Standard Cover Order**
    '''
    if len(X1) != len(X2):
        if len(X1) > len(X2):
            return -1
        else:
            return 1
    elif support(X1) != support(X2):
        if support(X1) > support(X2):
            return -1
        else:
            return 1
    else:
        for i in range(len(X1)):
            if X1[i] != X2[i]:
                if X1[i] < X2[i]:
                    return -1
                else:
                    return 1
        return 0

In [19]:
def strset2intset(strset: set):
    ret = np.array([])
    for el in strset:
        ret = np.append(ret,str2vec(el))
    return set(np.ravel(ret))

In [20]:
def getCover(txn: list, CT):
    '''
    <algorithm 2>

    Get the standard cover of a transaction.
    
    @param txn: A transaction
    @param CT:  code table CT
    @return: a set, standard cover
    CT = { (str)"items":(int)code, ...}
    '''
    # get the standard code
    code = set()
    
    # S ← smallest element X of CT in Standard Cover Order for which X ⊆ t
    for k,v in CT.items():
        if set(str2vec(k)).issubset(set(txn)):
            # add string to set <can this be better?>
            code.add(k)
            # code = set(str2vec(k))
            break
    # if t \ S = ∅ then
    if len(set(txn) - strset2intset(code)) == 0:
        return code
    else:
        # Res ← {S} ∪ StandardCover(t \ S, CT )
        return code.union(getCover(list(set(txn) - strset2intset(code)), CT))

In [21]:
# test = getCover(txns, SCT)
# test

In [22]:
def properPowerset(iterable):
    '''
    powerset([1,2,3]) --> (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    '''
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(2,len(s)+1)))

In [23]:
def getCandidateSet(txns: list):
    """
    
    candidate set F = D in Standard Candidate Order.

    @param txns: database / list of transactions
    @return: standard code table SCT
    SCT = { itemName:(code,frequency) , ...}
    """
    # get the unique items in transactions
    d = OrderedDict()
    for tx1 in txns:
        # calculate support
        for ps in properPowerset(tx1):
            key = vec2str(ps)
            if(key not in d):
                d[key] = 1
            else:
                d[key] += 1
    
    d = sortSCadO(d)

    return d


F = getCandidateSet(txns)

In [24]:
F

OrderedDict([('1 8', 3),
             ('1 8 17', 2),
             ('0 17', 2),
             ('0 21', 2),
             ('1 17', 2),
             ('3 17', 2),
             ('3 59', 2),
             ('4 8', 2),
             ('4 49', 2),
             ('8 17', 2),
             ('10 11', 2),
             ('10 37', 2),
             ('17 38', 2),
             ('17 52', 2),
             ('23 135', 2),
             ('0 1 6 8 17 24 52 58', 1),
             ('4 16 18 39 49 68 79 80', 1),
             ('17 18 23 38 52 113 135 138', 1),
             ('0 1 6 8 17 24 52', 1),
             ('0 1 6 8 17 24 58', 1),
             ('0 1 6 8 17 52 58', 1),
             ('0 1 6 8 24 52 58', 1),
             ('0 1 6 17 24 52 58', 1),
             ('0 1 8 17 24 52 58', 1),
             ('0 6 8 17 24 52 58', 1),
             ('0 17 21 37 55 67 68', 1),
             ('1 3 8 17 36 39 137', 1),
             ('1 4 8 9 13 33 49', 1),
             ('1 6 8 17 24 52 58', 1),
             ('4 16 18 39 49 68 79', 1),
   

In [25]:
def LDCT(txns, encoding, xdict):
    ret = 0
    for txn in txns:
        arg = vec2str(txn)
        for X in encoding[arg]:
            ret += np.log(sum(xdict.values())/xdict[X])

    return ret
            

In [26]:
def LCTD(txns, encoding, xdict):
    # L(CT | D) = sum                         < L(code_ST(X))(->????!!!!!) + L(code_CT(X)) >
    #                (for X ∈ CT :usage(X)!=0)  
    ret = 0
    for X in xdict.keys():
        LcodeSTX = 0
        for item in X.strip().split():
            # print(item)
            LcodeSTX += np.log(sum(SCT.values())/SCT[item])
        ret += LcodeSTX
        ret += np.log(sum(xdict.values())/xdict[X])

    return ret

In [27]:
def getL(txns: list, encoding, xdict):
    return LDCT(txns, encoding, xdict) + LCTD(txns, encoding, xdict)

In [28]:
def coverall(CT, txns: list):
    '''
    @return a dict of items and their cover, a dict of cover stats {X: (usage)}

    @definition usage: the number of transactions that contain X in their cover
    '''
    xCov = OrderedDict()
    covDict = OrderedDict()
    for txn in txns:
        cov = getCover(txn, CT)
        covDict[vec2str(txn)] = cov
        for X in cov:
            if X not in xCov:
                xCov[X] = 1
            else:
                xCov[X] += 1
    
    return covDict, xCov

In [29]:
# a,b = coverall(SCT, txns)

In [30]:
# a

In [37]:
def krimp(txns: list, F, SCT):
    """
    <algorithm 3>
    Our main KRIMP algorithm
    @param txns: database / list of transactions
    @param F: candidate set F
    @param SCT: standard code table SCT
    @return: compressed code table CT
    """

    CT = SCT.copy()
    for key, value in F.items():
        # CTc ← (CT ∪ F) in Standard Cover Order
        CTc = CT.copy()
        # [TODO] Modify to add key at proper position
        CTc[key] = value
        CTc = sortSCadO(CTc)
        # cover every txn and get cover stats
        encoding_ctc, dictx_ctc = coverall(CTc, txns)
        encoding_ct, dictx_ct = coverall(CT, txns)
        # print(encoding_ctc)
        left = getL(txns, encoding_ctc, dictx_ctc)
        right = getL(txns, encoding_ct, dictx_ct)

        if left< right:
            print(key,':',value)
            print(left,' ',right)
            CT = CTc

    return CT

In [38]:
ans = krimp(txns, F, SCT)

784.1222759791967   798.7896878871752
762.92653989085   784.1222759791967
741.8385321252778   762.92653989085
720.8611292530331   741.8385321252778
699.9973615592241   720.8611292530331
689.6090829250745   699.9973615592241
679.2504254465484   689.6090829250745
668.921816885477   679.2504254465484
658.6236971873383   668.921816885477
648.3565189981655   658.6236971873383
638.1207482106156   648.3565189981655
627.9168645412766   638.1207482106156
617.74536214144   627.9168645412766


In [33]:
ans

OrderedDict([('10', 5),
             ('17', 5),
             ('1', 4),
             ('3', 4),
             ('4', 4),
             ('8', 4),
             ('0', 3),
             ('18', 3),
             ('21', 3),
             ('37', 3),
             ('39', 3),
             ('23 135', 2),
             ('11', 2),
             ('22', 2),
             ('23', 2),
             ('38', 2),
             ('49', 2),
             ('52', 2),
             ('58', 2),
             ('59', 2),
             ('68', 2),
             ('70', 2),
             ('79', 2),
             ('135', 2),
             ('2 72 128', 1),
             ('9 13 33', 1),
             ('15 95 162', 1),
             ('44 69 77', 1),
             ('6 24', 1),
             ('16 80', 1),
             ('25 82', 1),
             ('36 137', 1),
             ('43 60', 1),
             ('55 67', 1),
             ('64 129', 1),
             ('113 138', 1),
             ('2', 1),
             ('6', 1),
             ('9', 1),
             ('1

In [34]:
st = '12 105'
st.strip().split()

['12', '105']